In [1]:
import pandas as pd
from pandasql import *
import numpy as np

In [2]:
df = pd.read_csv('RUS.csv')
htos = pd.read_csv('htos.csv')

In [3]:
df.columns

Index(['Year', 'Reporter Code', 'Trade Flow Code', 'Partner Code',
       'Classification', 'Commodity Code', 'Quantity Unit Code',
       'Supplementary Quantity', 'Netweight (kg)', 'Value', 'Estimation Code'],
      dtype='object')

In [4]:
cols = [col for col in df.columns if col not in ['Reporter Code', 'Trade Flow Code', 'Estimation Code', 'Supplementary Quantity',
                                                'Classification', 'Partner Code', 'Quantity Unit Code']]
df2 = df[cols]

In [5]:
df2.columns

Index(['Year', 'Commodity Code', 'Netweight (kg)', 'Value'], dtype='object')

In [6]:
df3 = df2.rename(columns = {
            'Commodity Code': 'Code',
            'Netweight (kg)': 'Netweight_kg',
            'Value': 'Value_us'
})

In [7]:
df3.head()

,Year,Code,Netweight_kg,Value_us
0,2011,01,NaN,553780836
1,2011,01,NaN,6698028
2,2011,0101,233078.0,2729948
3,2011,0101,102100.0,654562
4,2011,010110,102953.0,1711722


In [8]:
df3.dtypes

Year              int64
Code             object
Netweight_kg    float64
Value_us          int64
dtype: object

In [9]:
df3['Code'] = df3['Code'].astype('str')
mask = (df3['Code'].str.len() == 6)
df4 = df3.loc[mask]

In [10]:
df4.columns

Index(['Year', 'Code', 'Netweight_kg', 'Value_us'], dtype='object')

In [11]:
htos.head()

,HS,GTAP,SAM53
0,100610,1,1
1,100620,1,1
2,100110,2,1
3,100190,2,1
4,100200,3,1


In [12]:
sqldf = PandaSQL()

In [13]:
df5 = sqldf("""
SELECT Code, SUM(Netweight_kg) AS Netweight_kg, SUM(Value_us) AS Value_us
FROM df4
GROUP BY Code;
""")

In [14]:
df5 = sqldf("""
SELECT df4.*, htos.SAM53
FROM df4
LEFT JOIN htos
ON df4.Code = htos.HS
""")

In [15]:
df5.head()

,Year,Code,Netweight_kg,Value_us,SAM53
0,2011,010110,102953.0,1711722,7.0
1,2011,010110,32680.0,554049,7.0
2,2011,010190,130125.0,1018226,7.0
3,2011,010190,69420.0,100513,7.0
4,2011,010210,42335707.0,319389080,7.0


In [16]:
df6 = sqldf("""
SELECT SAM53, SUM(Netweight_kg) AS Netweight_kg, SUM(Value_us) AS Value_us,
SUM(Value_us)/SUM(Netweight_kg) AS Value_per_kg
FROM df5
GROUP BY SAM53
""")

In [17]:
df7 = df6.dropna(how = 'any')
df7['SAM53'] = df7['SAM53'].astype('int')
df7.head()

C:\Users\saily\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SAM53,Netweight_kg,Value_us,Value_per_kg
1,1,1.869695e+10,4633860826,0.247840
2,2,1.064064e+10,9233612626,0.867769
3,3,1.497056e+09,1064987134,0.711388
4,5,8.801104e+07,195924728,2.226138
5,6,8.125691e+08,3464675758,4.263854
